# Final Project for Numerical Methods
by Eszter Vági, Silke Kaiser and Eric Kolibacz

# A* Search Algorithm


<a id='index-0'></a>

## Contents
 
  - [1. Overview](#1.-Overview)  
      - [1.1. Motivation](#1.1.-Motivation)  
  - [2. A* Search Algorithm](#2.-A*-Search-Algorithm)  
      - [2.1. Outline of the Problem](#2.1.-Outline-of-the-Problem)  
      - [2.2. The functioning of the algorithm](#2.2.-The-functioning-of-the-algorithm)            
          - [2.2.1. Basic idea](#2.2.1.-Basic-idea)    
          - [2.2.2. The heuristic function $ h(n) $](#2.2.2.-The-heuristic-function-$-h(n)-$) 
      - [2.3. Implementation in Julia](#2.3.-Implementation-in-Julia) 
          - [2.3.1. Creating the Setting to save the collected information of the nodes](#2.3.1.-Creating-the-Setting-to-save-the-collected-information-of-the-nodes)   
          - [2.3.2. G-Function](#2.3.2.-G-Function)        
          - [2.3.3. H-Function](#2.3.3.-H-Function)        
          - [2.3.4. Finding the neighbouring positions](#2.3.4.-Finding-the-neighbouring-positions)       
          - [2.3.5. Finding the best position to investigate](#2.3.5.-Finding-the-best-position-to-investigate)      
          - [2.3.6. Implementing the algorithm](#2.3.6.-Implementing-the-algorithm)        
      - [2.4. Feeding an Example](#2.4.-Feeding-an-Example)           
          - [2.4.1. Simple Example](#2.4.1.-Simple-Example) 
          - [2.4.2. Medium Example](#2.4.2.-Medium-Example) 
          - [2.4.3. Difficult Example](#2.4.3.-Difficult-Example)
          - [2.4.4. Unsolvable Example](#2.4.4.-Unsolvable-Example)
  - [3. Conclusion](#3.-Conclusion)     
      - [3.1. Comparison of the H-functions](#3.1.-Comparison-of-the-H-functions)               
      - [3.2. Further extension possibilities](#3.2.-Further-extension-possibilities)      
  - [Sources](#Sources)

 

## 1. Overview

In this notebook, we present the A* search algorithm, provide an extensive explanation about how it works and provide a code to solve a sliding puzzle. All further mathematical/theoretical explanations are based on chapter 24 of [Cormen et al. (2009)](https://mitpress.mit.edu/books/introduction-algorithms-third-edition).

The shortest path problem describes the issues of finding the shortest path between two nodes. It is a classical problem in mathematics and computer science with applications in economics (sequential decision making, analysis of social networks, etc.), operations research and transportation or robotics and artificial intelligence. There are plenty of algorithms for solving the shorthest path problem. For instance, the Dijkstra's algorithm was the first that addressed this issue in 1959. The A* algorithm is an alternative approach.

The A* search algorithm can be considered as a informed variation of the Dijkstra's algorithm, as it uses information about path cost and also uses heuristics to find the solution. The latter is one of the main advantages that A* provides compared to Dijkstra's as the use of heuristics is meant to speed up the search. It is also one of the most successful search algorithms to find the shortest path between nodes or graphs. The A* search algorithm was first described in 1968 by Peter Hart, Nilds J. Nilsson and Bertram Raphael.

### 1.1. Motivation

Both the A* and Dijkstra's algorithms can be considered as special cases of dynamic programming. Thus, they are very powerful optimisation techniques that can be applied on a large number of fields, including Economics.

One of the fields where shortest path algorithms are employed in Economics is the field of networks, which we are especially interested in. For example, there is a vast metaliterature on the importance of citations and affiliations with journals. Investigating those  networks is also approached by economists. Among others, [Jackson and Rogers (2005)](https://doi.org/10.1162/jeea.2005.3.2-3.617) investigate networks where agents profit from indirect relationships. However, networks have also been considered in a more spatial manner by economists, for example by [Chen et al. (2013)](https://doi.org/10.1007/s11067-012-9175-1) who compare different shortest path algorithms in the face of travel times in road networks, among them the A* algorithm.
Beyond that, the A* algorithm stroke us as an efficient approach that can be used in multifaceted ways. For instance, it serves to solve shortest path problems in Game Theory. To give an example, [Chandrashekar & Narahari (2005)](https://doi.org/10.1007/11600930_8) develop shortest path cooperative games where agents have to go from a starting node to a specified target node in a directed acyclic network. The authors use this framwork to investigate the willingness to share the surplus earned from finding the shortest path.

Furthermore, we regard the A* search algorithm as an optimal extension to the QuantEcon notebook on the [shortest path](https://julia.quantecon.org/dynamic_programming/short_path.html) problem. We were motivated to see how this problem can be extended, and what could be a better way to find the shortest path; eventually emplyoing the learning outcomes in our subsequent PhDs and work.


## 2. A* Search Algorithm

### 2.1. Outline of the Problem

In this notebook, we solve a so-called sliding puzzle that consists of several parts in a frame with one empty space, such that only one piece can be moved at a time. The elements are normally in the wrong order, and the goal is to put them in the right order. In such riddles, sometimes the goal is to reveal an image by moving their parts in the right positions, which is a game that some might know from their childhood.

With the help of the A* algorithm, we will try to find the most efficient way to solve such a puzzle and to reach the goal. An example is provided in the images below (with the starting position on the left and the goal position on the right).

<img src="Sliding_Puzzle.JPG" style="width:50%;">



The A* algorithm is a search algorithm in the sense that it aims to sort its elements with respect to the goal nodes, thereby minimizing the cost (which can be defined as the shortest distance, least time etc.). The idea is that intitially only the starting and goal nodes are known. Then the algorithm builds up a tree, originating at the starting node finding the least costly path to the goal. 

In comparison to other search algorithms, it uses a heuristic function to search for the solution which allows a faster determination. Thereby, the algorithm is complete and optimal - meaning that if an optimal solution exists, it is always found. However, it is important to note that A* is complete and optimal only if an admissible heuristic function is used. This point is going to be developed in greater detail later on.

### 2.2. The functioning of the algorithm

### 2.2.1. Basic idea

At the beginning, only the starting and goal nodes are known.
The algorithm proceeds, by splitting the nodes into three different categories: known, unknown and completely investigated nodes. Unknown nodes are those that the algorithm has not found yet during the search. Known nodes are those that have been found by the algorithm and to which a path from the starting node is known. Each known node gets an $f(n)$ value assigned:
<br>
<br>
<center> $ f(n) = g(n) + h(n) $ </center>
<br>

Thereby, $ g(n) $ denotes the cost from the starting node to node $ n $ and $ h(n) $ is a function estimating the cheapest cost from $ n $ to the goal node. The A* algorithm starts by analyising those nodes that are likely to reach their goal node first (e.g. with the lowest $ f(n) $ value assigned). 

Completely investigated nodes are those to which the shortest path has been found. Those nodes are saved on the so called $\textit{closed list}$. The other known nodes are saved on the $\textit{open list}$. Every known and completely investigated node indicates (via an "arrow") from which previous node it was reached. 

A node becomes completely investigated, once all the nodes that can be reached from it are included in the open list (all "following nodes" are found). If the following nodes are already in the closed list, they remain in that list and are not investigated again. If a following node is in the open list, it is investigated whether the node is being reachable in a less costly path via this node than in comparison to the node through which it was found previously. If this is the case, the "arrow" of the node is updated. 

The A* search algorithm comes to an end, once it has reached the goal node and the goal node is completely investigated. Then tracing back the arrows from the goal, the least costly path is found. The algorithm also terminates, if the open list becomes empty - under those circumstances no path to the goal node exists. Thereby, the function $ h(n) $ is a key element of the algorithm; if it is admissible, A* will always return the least-cost path.


### 2.2.2. The heuristic function $ h(n) $

As mentioned earlier, A* is complete and optimal only if an admissible heuristic function is used. This means that the real cost to reach goal node from node $ n $ should be smaller than or equal to $ h(n) $. Hence, the heuristic function should never overestimate the cost. Otherwise, the A* algorithm is no longer complete and optimal.

The heuristic function can be defined in several ways. The most frequent definition is that of the beeline from a node to the goal node. This is due to the fact that via triangular inequality, the distance is never over estimated. The triangular inequality states that for any triangle, the sum of the lengths of any two sides must be greater than or equal to the length of the remaining side. Considering the sliding puzzle that we will investigate: As we can move the elements of the puzzle only vertically and horizontally, it is assured that the length of a straight line that goes from the starting node to the goal node is less than or equal to the actual distance the element has to "travel" to reach its final position.

Other typical examples of admissible heuristic functions are the "Hamming distance" and the "Manhattan distance". The former takes into account how many elements are in wrong positions, the latter considers the number of moves the elements have to take to reach their final positions.

It is also important to note that sometimes the admissibility criterion of the heuristic function is relaxed to speed up the algorithm. Such heuristic functions are called "ε-admissible", and the A* algorithms implementing such heuristics are usually referred to as "A* Epsilon". Using this approach, we approximate the shortest path to speed up the search at the expense of optimality by relaxing the admissibility criterion. The idea is that we can guarantee that the solution path is no worse than (1 + ε) times the optimal solution path. 

In the next section, we will provide both admissible (Hamming distance and Manhattan distance) and ε-admissible heuristic functions that will be implemented by our A* algorithm.



### 2.3. Implementation in Julia

### 2.3.1. Creating the Setting to save the collected information of the nodes
The A* algorithm works with three types of nodes; known, unknown and completely investigated nodes. For each of those nodes, information has been collected and will be eventually changed through the algorithm.

Thus, we first create a so-called dictionary in which we can save the information on the individual nodes. A dictionary is a data structure in Julia that is somewhat similar to an array or a set but inherits characteristics useful for our problem. A dictionary associates unique identifiers, called the keys, to corresponding values. In our case, the unique identifier will be the position (which is equal to a node in the previous theoretical explanation) - which is a nice characteristic as we want to list every position only once in our list and eventually update its respective information.

We will work with the puzzle and understand each combination of elements as a position. For example, the image given above has a Starting position of [8, 1, 7, 4, 5, 6, 2, 0, 3] and a Goal position of [0, 1, 2, 3, 4, 5, 6, 7, 8]. Note that the missing, or empty piece, is denoted by a 0. 

Staying within this example, the positions that can be discovered from the Starting position are the followings:

- [8, 1, 7, 4, 0, 6, 2, 5, 3] when the piece 5 is moved downwards,
- [8, 1, 7, 4, 5, 6, 0, 2, 3] when the piece 2 is moved to the right,
- [8, 1, 7, 4, 5, 6, 2, 3, 0] when the piece 3 is moved to the left.


Thus, the dictionary would take the exemplary form (with H-values being only exemplary):



| Position  | H-Funtion-Value | G-Function-Value | Parent|
| --- | --- | --- |---|
| [8, 1, 7, 4, 5, 6, 2, 0, 3] | 29.9 | 0 | ---|
| [8, 1, 7, 4, 0, 6, 2, 5, 3]| 13.8| 1 |[8, 1, 7, 4, 5, 6, 2, 0, 3]|
| [8, 1, 7, 4, 5, 6, 0, 2, 3]| 15.3 | 1 |[8, 1, 7, 4, 5, 6, 2, 0, 3]|
| [8, 1, 7, 4, 5, 6, 2, 3, 0]| 20.9| 1 |[8, 1, 7, 4, 5, 6, 2, 0, 3]|

As mentioned above, nodes are always discovered from a previous node. We will call this previous node the parent.

The following creates an "empty" dictionary. We could also use "any" instead of the arrays and floats, but we decided to detail the types in order to make the code run faster (as seen in class). The first Array{} denotes the Position, H and G-values are Float numbers and the parent is again an array in itself. 

In [1]:
hs = Dict{Array{}, Float64}() 
gs = Dict{Array{}, Float64}()
parents = Dict{Array{}, Array{}}()

Dict{Array,Array} with 0 entries

The following function allows to make entries into the dictionary. Meaning to store the respective information of every node we find. Again, the parent denotes the node from which the current node was found.

In [2]:
function write_entry(position, h, g, parent)
    hs[position] = h
    gs[position] = g 
    parents[position] = parent 
end

write_entry (generic function with 1 method)

### 2.3.2. G-Function

As explained above, the G-function denotes the cost it has taken to reach the current node. Such a G-function can measure time or distance. In our example, we define the G-function as a function that gains +1 for every switch of the puzzle pieces. Meaning for example to go from Start [8, 1, 7, 4, 5, 6, 2, 0, 3] to [8, 1, 7, 4, 0, 6, 2, 5, 3], the G-value increases by 1. We will later normalise the initial G-value to 0.

Thus, we can simply add +1 to the G-value of the parent to obtain the G-value of the current node.

In [3]:
function calculate_G(G_of_parent)
    return G_of_parent+1
end    

calculate_G (generic function with 1 method)

### 2.3.3. H-Function

Defining the H-function is much more complicated. Thus, we create several different alternatives and will compare their efficiency later. The following function selects among the different H-functions we propose. The H-function used will later be chosen in A*.



In [50]:
function calculate_H(position, option)
    if option == 1
        h = H_HammingDistance(position)
    end
    if option == 2
        h = H_ManhattanDistance(position)
    end
    if option == 3
        h = H_HammingDistance_weighted_nonadmissible(position)
    end
    if option == 4
        h = H_ManhattanDistance_alternative_nonadmissible(position)
    end
    if option == 5
        h = H_ManhattanDistance_weighted_nonadmissible(position)
    end
    return h
end

calculate_H (generic function with 1 method)

In the following, we code the different possibilities for the H-function.  We begin by two admissible H-functions, the Hamming and the Manhattan Distance and then introduce three variations of those distances that are non-admissible.

1. Hamming Distance Admissible

One possible definition is such that the value of the H-function indicates how many elements are in a wrong position. As mentioned earlier, this approach is also called the "Hamming distance" and it is defined as:
$$x=(x_1,...,x_n)$$
$$y=(y_1,...,y_n)$$
with the Hamming Distance
$$ \Delta(x,y):=|j\in \{ 1,...,n \} | x_j \neq y_j \} |$$


In our example, the highest value the H-function can take is 9 (if all elements are in a wrong position) to 0 (if all elements are in the right position, i.e. we are at the goal position). The value of the function decreases the closer we come to the solution.


In [5]:
function H_HammingDistance(position)
    h = size(goal,1) 
    for (g,p) in zip(goal, position) 
        if g == p
            h -= 1
        end
    end
    return h 
end

H_HammingDistance (generic function with 1 method)

2. Manhattan Distance Admissible

The idea behind the Manhattan distance is that we should calculate the distance (defined in number of moves) between the goal and current position for each element, and add them all together. The higher this Manhattan distance is, the further away we are from our goal. This is a proven admissible H-function. The Manhattan Distance is defined as follows:
$$x=(x_1,...,x_n)$$
$$y=(y_1,...,y_n)$$
with the Manhattan Distance
$$ \Delta(x,y):=||x-y ||_1=\sum\limits_{i=1}^{n}|x_i-y_i|$$

In a plane like our puzzle, this distance between the point ($x_1,x_2$) and ($y_1,y_2$) is
$$ |x_1-y_1|+|x_2-y_2|$$

To produce the Manhattan distance, we need to consider our positions as a 3x3 plane, and not as a vector. We imagine the matrix and denote the x,y$\in{1,2,3}$. Based on this matrix, we subtract the goal position minus the current position of the goalmatrix and take the sum of the absolute values to obtain the individual Manhattan Distances for each puzzle piece and sum up all those values to obtain the Manhattan Distance. We divide the final distance by two: imagine only 0 and 1 remain in the wrong positions. The distance would claim this as "remaining path=2", thus we divide by two.

When reading the code, one has to keep in mind as learned in class, that the indices in Julia start with 1. Thus, the puzzle is in the right position if at indice 1 is the piece 0, at indice 2 the piece 1, at indice 3 the piece 2 etc. See below for a graphical representation of the indices. (Note that we denoted the array in a vector shape, not in a matrix. Thus, the numbering continues over the lines.)


|   |  |  | 
| --- | --- | --- |
| 1 | 2|3| 
| 4| 5| 6 |
| 7| 8 | 9 |

In [6]:
function H_ManhattanDistance(position)
    h=0
    
    # We code the goal matrix as a vector (the first is the x value, the second the y value in a 0-2 x 0-2 plane.)
    goalmatrix=[(0,0),(1,0),(2,0),(0,1),(1,1),(2,1),(0,2),(1,2),(2,2)]
    for i in 0:8
        index_i=findfirst(isequal(i),position)
        matrix_i=goalmatrix[index_i]
        h+=sum(abs.(matrix_i.-goalmatrix[(i+1)]))
    end    
    return h/2
end

H_ManhattanDistance (generic function with 1 method)

3. Weighted Hamming Distance Non-Admissible

A Hamming Distance normally is admissible for a sliding puzzle as the total number of moves to be made to reach the goal, is at least the number of misplaced puzzle pieces. Thus, in the framework of a weighted Hamming Distance, where each misplaced puzzle piece is weighted by $\geq$1, the function becomes $\epsilon$-admissible. This shall serve to eventually compare the performance of the H-functions and the results they deliver, depending on their admisibility and their respective weight in comparison to the G-function.

Example of overestimation:

Imagine all puzzle pieces are in their respective right positions besides puzzle piece "0" and puzzle piece "3" which are alternated. From this position, there would be one remaining move to the goal position. However, this H-function estimates the remaining distance as 2.

In [9]:
function H_HammingDistance_weighted_nonadmissible(position)
    h = size(goal,1) + 24 
        if goal[1] == position[1]
            h -= 1
        end
         if goal[2] == position[2]
            h -= 1
        end
         if goal[3] == position[3]
            h -= 2
        end
         if goal[4] == position[4]
            h -= 1
        end
         if goal[5] == position[5]
            h -= 2
        end
         if goal[6] == position[6]
            h -= 5
        end
         if goal[7] == position[7]
            h -= 2
        end
         if goal[8] == position[8]
            h -= 5
        end
        if goal[9] == position[9]
            h -= 8
        end
    return h
end

H_HammingDistance_weighted_nonadmissible (generic function with 1 method)

4. Manhattan Distance Alternative  Non-Admissible

The next function we propose is inspired by the "Manhattan distance", but it is slightly different. 

We implement a simplistic version of the Manhattan Distance, where we take the absolute value of the difference between the current position and the goal for each element and we add them up. The way it is defined makes this H-function ε-admissible, as it tends to overestimate the actual distance sometimes.

Example of overestimation:

Imagine a position with number 8 on the top left corner (where goal is 0), then:
 - Calculated distance by this function = 8 <br>
 - Actual distance = 4

Thus, we can see that the calculated distance is higher than the actual distance. Note that in some cases, the opposite could happen where a distance is underestimated (i.e. number 6 at the place where goal is number 5: calculated distance is 1 while the actual distance is 3).

In [10]:
function H_ManhattanDistance_alternative_nonadmissible(position)  
    for (g,p) in zip(goal, position) 
        h=broadcast(abs, goal[1] - position[1])+
        broadcast(abs, goal[2] - position[2])+
        broadcast(abs, goal[3] - position[3])+
        broadcast(abs, goal[4] - position[4])+
        broadcast(abs, goal[5] - position[5])+
        broadcast(abs, goal[6] - position[6])+
        broadcast(abs, goal[7] - position[7])+
        broadcast(abs, goal[8] - position[8])+
        broadcast(abs, goal[9] - position[9])  
        return h 
    end 
end

H_ManhattanDistance_alternative_nonadmissible (generic function with 1 method)

5. Manhattan Distance Weighted Non-Admissible

In a similar fashion to the weighted Hamming Distance, we include a weighted Manhattan Distance where the weights do not allow the function to be admissible. However, this will allow a stronger weighting of the H-function relative to the G-function.

In [55]:
function H_ManhattanDistance_weighted_nonadmissible(position)
    h=0
    
    # We code the goal matrix as a vector (the first is the x value, the second the y value in a 0-2 x 0-2 plane.)
    goalmatrix=[(0,0),(1,0),(2,0),(0,1),(1,1),(2,1),(0,2),(1,2),(2,2)]
    weight=[0.5,1,3,1,3,5.5,3,5.5,8]
    for i in 0:8
        index_i=findfirst(isequal(i),position)
        matrix_i=goalmatrix[index_i]
        h+=sum(abs.(matrix_i.-goalmatrix[(i+1)]))*weight[index_i]
    end    
    return h/2
end

H_ManhattanDistance_weighted_nonadmissible (generic function with 1 method)

### 2.3.4. Finding the neighbouring positions

In the following paragraph, we define how the algorithm finds new nodes. In order to do this, we defined several subfunctions. So let's consider each of them to understand the process.

1. The Switch Around Function

The switch_around function states that the element 0 can be switched with any other puzzle piece. However, here we have to work with indices (as the puzzle pieces with their respective numbers can be at any point of the puzzle). 

In [11]:
function switch_around(position,i,j)
    new_position = copy(position)
    new_position[j] = position[i]
    new_position[i] = position[j]
    return new_position
end

switch_around (generic function with 1 method)

2. The Add Neighbours Function

The add neighbours function considers how many pieces can be switched with the "puzzle element 0". Meaning if the 0 is in the middle at the bottom as in the example above, it can be switched with 3 possible pieces (2, 5 or 3). If the 0 is in the middle, it can be switched with four possible puzzle pieces, etc. Julia recognizes automatically which of the functions has to be used, depending on the number of elements given to the function.


In [12]:
# Add neighbours function for 2 possible switches
function add_neighbours(position, index, a, b)
    np1 = switch_around(position,index,a)
    np2 = switch_around(position,index,b) 
    return [np1, np2] # we return the IDs of the new positions we have just added   
end

# Add neighbours function for 3 possible switches
function add_neighbours(position, index, a, b, c)
    np1 = switch_around(position,index,a)
    np2 = switch_around(position,index,b)
    np3 = switch_around(position,index,c)
    return [np1, np2, np3]
end

# Add neighbours function for 4 possible switches
function add_neighbours(position, index, a, b, c, d)
    np1 = switch_around(position,index,a)
    np2 = switch_around(position,index,b)
    np3 = switch_around(position,index,c)
    np4 = switch_around(position,index,d)
    return [np1, np2, np3, np4]
end

add_neighbours (generic function with 3 methods)


3. Finding Neighbours Function

The finding neighbours function now considers all cases where the "0" puzzle piece might be and depending on this what could be the following nodes that can be found from this respective position (i.e. find the children of the parent). As previously mentioned, we have to work with the indices and not with the puzzle piece itself. Thus we look for the "index_zero" and which value it takes. For example if index_zero==1, e.g. the empty puzzle piece is in the left upper corner, we can switch it with the position with index 2 and with the position with index 4. 

In [13]:
function finding_neighbours(position)
    # We find the first 0 in the array (position) and note its index. As there is only one 0 per position 
    # -->(only one empty puzzle piece)
    index_zero=findfirst(isequal(0),position) 
    if index_zero==1
        neighbours = add_neighbours(position,index_zero,2,4) 
    end
    if index_zero==3
        neighbours = add_neighbours(position,index_zero,2,6)
    end
    if index_zero==7
        neighbours = add_neighbours(position,index_zero,4,8)
    end
    if index_zero==9
        neighbours = add_neighbours(position,index_zero,6,8)
    end
    if index_zero==2
        neighbours = add_neighbours(position,index_zero,1,3,5)
    end
    if index_zero==4
        neighbours = add_neighbours(position,index_zero,1,5,7)
    end
    if index_zero==6
        neighbours = add_neighbours(position,index_zero,3,5,9)
    end
    if index_zero==8
        neighbours = add_neighbours(position,index_zero,5,7,9)
    end
    if index_zero==5
        neighbours = add_neighbours(position,index_zero,2,4,6,8)
    end
    return neighbours
end


finding_neighbours (generic function with 1 method)

### 2.3.5. Finding the best position to investigate

As mentioned in the explanation of the A* search algorithm, the algorithm always investigates the node $ n $ first that yields the most promising F-value (as low as possible):

$ f(n) = g(n) + h(n) $

The following function takes up the job to pick this most promising point out of a list including several positions. The function does this as follows: 

We define two variables: current_min_f (the minimum of $ f(n) $) and current_min_position (the position with the lowest F-value currently found). We let both of these variables start at infinity as we are looking for the minimum. We then go through the list of positions and note the position with the lowest F-value. The function returns this position.

In [14]:
function determine_minimum_f(list_of_positions) 
    current_min_f = Inf
    current_min_position = Inf
    for p in list_of_positions 
        if gs[p] + hs[p] < current_min_f 
            current_min_f = gs[p] + hs[p]
            current_min_position = p 
        end
    end
    return current_min_position
end

determine_minimum_f (generic function with 1 method)

### 2.3.6. Implementing the algorithm

We are slowly advancing to the end of this algorithm. The following function uses the previous functions to actually build the complete A* algorithm. 

We will explain this function step by step.

1. Naming function and Open / Closed list

We begin by naming the function and stating as the input the starting position and the goal position we want to achieve. The open and closed list are lists of the different positions. Their properties and how points move between those lists have been explained previously. Those lists only contain the positions, their respective H, G and parent-positions remain noted in the dictionary. The lists are defined as sets, which is similar to an array. However, repetitions are not allowed in sets, which is convenient, as we do not want to have the same positions named several times per set (open or closed list).

2. Write Entry Function

We add the starting point to the dictionary. Thereby, we take its position, calculate its H-value, normalise its G-value to 0 and denote its parent by an array of [-1,-1,-1,-1,-1,-1,-1,-1,-1]. This position does obviously not exist - this is done as the starting position has no parent; thus we use a fictionary array.

3. Add Entry to Open List

We continue by adding the starting-position "start" to the open list. The command push! is needed to add something to a set. Later we will also add entries to an array, for which we need the command "append!", more on this below.

4. The Open List is Non-Empty

As explained before, if the algorithm reaches the point at which the open list is empty, the goal point has been found or the algorithm has to come to the conclusion that no path can be found. 
If the list is non-empty we do the following, as described in this equation: We determine the most promising point from the open list (determine_minimum_f). To see how the algorithm advances (and to double check if it works) we print all those points, together with its G and H-values. We also print the length of the open and closed list. We also include an option how many results shall be printed (display_every_ith_output) as this can make the computation much faster for difficult examples. If the position that is investigated is equal to the goal point, the algorithm has come to its end. Then, as explained previously, we want to determine this "shortest path". We do this by going back through all the parent-relationships, meaning we note the path from goal to start. We do this as long as the parent position (at its first index) is greater than -1, meaning until the start (whose parent was an array of -1s). In the end, we reverse all entries in "path" to actually obtain the path from start --> goal.

5. Updating the lists

The node then becomes a completely investigated node, meaning it is deleted from the open list and added to the closed list.

6. Finding nodes that are already in the Closed List / Open List

As explained before, if a node is found that is already in the closed list, this node will be ignored. If they are in the open list, we evaluate their G-value: have we found a shorter path to reach this node or is the previously indicated parent (and the corresponding path) shorter. If the new path is shorter, we update the G-value and the parent (eg. change the arrows). 

7. Neighbours that are not in the Open or Closed List

If a new node is found, that is in neither list so far, we add it to the open list.

8. Open list is empty

If the openlist is empty, there exists no possible path from start to goal.

In [15]:
# 1. Naming function and Open / Closed list
function A_star(start, goal, option,display_every_ith_output=1) 
    openlist= Set{Array{Int64}}() 
    closedlist= Set{Array{Int64}}()
    
    #2. Write Entry function
    write_entry(start, calculate_H(start, option), 0, [-1,-1,-1,-1,-1,-1,-1,-1,-1]) 

    # 3. Add Entry to Open List 
    push!(openlist, start) 
    
    #4. The Open List is Non-Empty
    while !isempty(openlist) 
        current_position = determine_minimum_f(openlist)
        if(size(collect(values(closedlist)),1)%display_every_ith_output == 0)
            println(current_position, " ", gs[current_position]," ", hs[current_position],
            " ",size(collect(values(openlist)),1)," ",size(collect(values(closedlist)),1))
        end
        if current_position == goal 
            path = [] 
            while parents[current_position][1] >= 0
                append!(path,[current_position]) 
                current_position = parents[current_position]
            end
            append!(path,[current_position])
            return reverse(path)
        end
        
        # 5. Updating the Lists
        delete!(openlist,current_position); 
        push!(closedlist, current_position);
        
        # 6. Finding Nodes that are already in the Closed List / Open List
        for neighbour in finding_neighbours(current_position) 
           
            if in(neighbour, closedlist)
                continue 
            end
            
            if in(neighbour, openlist)
                tmp_g = calculate_G(gs[current_position])
                if tmp_g < gs[neighbour]
                    gs[neighbour] = tmp_g
                    parents[neighbour]=current_position
                end
                continue
            end
            
            #7. Neighbours that are not in the Open or Closed List
            push!(openlist, neighbour) # we simply add the neigbour_Id to the openlist
            write_entry(neighbour, calculate_H(neighbour, option), calculate_G(gs[current_position]), current_position)
        end        
    end
    
    #8. The Open list is empty
        while isempty(openlist)
            return "There exists no path." # The puzzle is unsolvable.
        end
end
        

A_star (generic function with 2 methods)

### 2.4. Feeding an Example

We are at the exciting point! Let's see what happens if we feed the algorithm with an example. We chose a start point (a mixed up puzzle) and want to sort it to the goal (here we use the sorting 0-8). We also print the final path found by the algorithm for each of the examples.

Please refer to the conclusion (section 3.) for an analysis of the findings.

### 2.4.1. Simple Example

We start by a very simple example where only the first row is moved towards the left in the puzzle. By this, we can see if the code produces usable results.

In [16]:
start1 = [1,2,0,3,4,5,6,7,8]
goal = [0,1,2,3,4,5,6,7,8]

9-element Array{Int64,1}:
 0
 1
 2
 3
 4
 5
 6
 7
 8

H-function Option 1 - Hamming Distance Admissible

In [17]:
@time final_path = A_star(start1,goal, 1)

[1, 2, 0, 3, 4, 5, 6, 7, 8] 0.0 3.0 1 0
[1, 0, 2, 3, 4, 5, 6, 7, 8] 1.0 2.0 2 1
[0, 1, 2, 3, 4, 5, 6, 7, 8] 2.0 0.0 3 2
  3.982006 seconds (3.09 M allocations: 157.506 MiB, 1.24% gc time)


3-element Array{Any,1}:
 [1, 2, 0, 3, 4, 5, 6, 7, 8]
 [1, 0, 2, 3, 4, 5, 6, 7, 8]
 [0, 1, 2, 3, 4, 5, 6, 7, 8]

H-function Option 2 - Manhattan Distance Admissible

In [18]:
@time final_path = A_star(start1,goal, 2)

[1, 2, 0, 3, 4, 5, 6, 7, 8] 0.0 2.0 1 0
[1, 0, 2, 3, 4, 5, 6, 7, 8] 1.0 1.0 2 1
[0, 1, 2, 3, 4, 5, 6, 7, 8] 2.0 0.0 3 2
  0.394994 seconds (4.33 k allocations: 237.475 KiB)


3-element Array{Any,1}:
 [1, 2, 0, 3, 4, 5, 6, 7, 8]
 [1, 0, 2, 3, 4, 5, 6, 7, 8]
 [0, 1, 2, 3, 4, 5, 6, 7, 8]

H-function Option 3 - Weighted Hamming Distance Non-Admissible

In [21]:
@time final_path = A_star(start1,goal, 3)

[1, 2, 0, 3, 4, 5, 6, 7, 8] 0.0 10.0 1 0
[1, 0, 2, 3, 4, 5, 6, 7, 8] 1.0 8.0 2 1
[0, 1, 2, 3, 4, 5, 6, 7, 8] 2.0 6.0 3 2
  0.150949 seconds (1.34 k allocations: 56.328 KiB)


3-element Array{Any,1}:
 [1, 2, 0, 3, 4, 5, 6, 7, 8]
 [1, 0, 2, 3, 4, 5, 6, 7, 8]
 [0, 1, 2, 3, 4, 5, 6, 7, 8]

H-function Option 4 - Alternative Manhattan Distance Non-Admissible

In [22]:
@time final_path=A_star(start1,goal, 4)

[1, 2, 0, 3, 4, 5, 6, 7, 8] 0.0 4.0 1 0
[1, 0, 2, 3, 4, 5, 6, 7, 8] 1.0 2.0 2 1
[0, 1, 2, 3, 4, 5, 6, 7, 8] 2.0 0.0 3 2
  0.072229 seconds (22.85 k allocations: 1.163 MiB)


3-element Array{Any,1}:
 [1, 2, 0, 3, 4, 5, 6, 7, 8]
 [1, 0, 2, 3, 4, 5, 6, 7, 8]
 [0, 1, 2, 3, 4, 5, 6, 7, 8]

H-function Option 5 - Weighted Manhattan Distance Non-Admissible


In [51]:
@time final_path=A_star(start1,goal, 5)

[1, 2, 0, 3, 4, 5, 6, 7, 8] 0.0 2.75 1 0
[1, 0, 2, 3, 4, 5, 6, 7, 8] 1.0 0.75 2 1
[0, 1, 2, 3, 4, 5, 6, 7, 8] 2.0 0.0 3 2
  0.586583 seconds (275.33 k allocations: 14.023 MiB)


3-element Array{Any,1}:
 [1, 2, 0, 3, 4, 5, 6, 7, 8]
 [1, 0, 2, 3, 4, 5, 6, 7, 8]
 [0, 1, 2, 3, 4, 5, 6, 7, 8]

### 2.4.2. Medium Example
We proceed with more difficult examples to actually compare the performance of the different H-functions. We begin by a medium difficult example to see in detail how the different H-functions perform.

In [23]:
start2=[1,3,4,2,7,5,6,8,0]

9-element Array{Int64,1}:
 1
 3
 4
 2
 7
 5
 6
 8
 0

H-function Option 1 - Hamming Distance Admissible

In [24]:
@time final_path = A_star(start2,goal, 1)

[1, 3, 4, 2, 7, 5, 6, 8, 0] 0.0 7.0 1 0
[1, 3, 4, 2, 7, 5, 6, 0, 8] 1.0 6.0 2 1
[1, 3, 4, 2, 0, 5, 6, 7, 8] 2.0 5.0 3 2
[1, 3, 4, 0, 2, 5, 6, 7, 8] 3.0 5.0 5 3
[1, 0, 4, 2, 3, 5, 6, 7, 8] 3.0 5.0 6 4
[0, 1, 4, 2, 3, 5, 6, 7, 8] 4.0 3.0 7 5
[0, 3, 4, 1, 2, 5, 6, 7, 8] 4.0 4.0 7 6
[1, 3, 4, 2, 5, 0, 6, 7, 8] 3.0 6.0 7 7
[1, 3, 4, 2, 7, 5, 0, 6, 8] 2.0 7.0 8 8
[2, 1, 4, 0, 3, 5, 6, 7, 8] 5.0 4.0 8 9
[2, 1, 4, 3, 0, 5, 6, 7, 8] 6.0 3.0 9 10
[1, 4, 0, 2, 3, 5, 6, 7, 8] 4.0 5.0 11 11
[1, 3, 4, 2, 7, 0, 6, 8, 5] 1.0 8.0 11 12
[1, 3, 4, 0, 7, 5, 2, 6, 8] 3.0 7.0 12 13
[0, 3, 4, 1, 7, 5, 2, 6, 8] 4.0 6.0 13 14
[1, 3, 4, 2, 0, 7, 6, 8, 5] 2.0 8.0 13 15
[1, 3, 4, 6, 2, 5, 0, 7, 8] 4.0 6.0 15 16
[1, 3, 0, 2, 7, 4, 6, 8, 5] 2.0 8.0 15 17
[1, 3, 0, 2, 5, 4, 6, 7, 8] 4.0 6.0 15 18
[3, 0, 4, 1, 2, 5, 6, 7, 8] 5.0 5.0 15 19
[1, 0, 3, 2, 5, 4, 6, 7, 8] 5.0 6.0 16 20
[0, 1, 3, 2, 5, 4, 6, 7, 8] 6.0 4.0 17 21
[3, 2, 4, 1, 0, 5, 6, 7, 8] 6.0 5.0 17 22
[2, 0, 4, 3, 1, 5, 6, 7, 8] 7.0 4.0 19 23
[0, 2, 4, 3, 

19-element Array{Any,1}:
 [1, 3, 4, 2, 7, 5, 6, 8, 0]
 [1, 3, 4, 2, 7, 5, 6, 0, 8]
 [1, 3, 4, 2, 0, 5, 6, 7, 8]
 [1, 0, 4, 2, 3, 5, 6, 7, 8]
 [1, 4, 0, 2, 3, 5, 6, 7, 8]
 [1, 4, 5, 2, 3, 0, 6, 7, 8]
 [1, 4, 5, 2, 0, 3, 6, 7, 8]
 [1, 4, 5, 0, 2, 3, 6, 7, 8]
 [0, 4, 5, 1, 2, 3, 6, 7, 8]
 [4, 0, 5, 1, 2, 3, 6, 7, 8]
 [4, 2, 5, 1, 0, 3, 6, 7, 8]
 [4, 2, 5, 1, 3, 0, 6, 7, 8]
 [4, 2, 0, 1, 3, 5, 6, 7, 8]
 [4, 0, 2, 1, 3, 5, 6, 7, 8]
 [0, 4, 2, 1, 3, 5, 6, 7, 8]
 [1, 4, 2, 0, 3, 5, 6, 7, 8]
 [1, 4, 2, 3, 0, 5, 6, 7, 8]
 [1, 0, 2, 3, 4, 5, 6, 7, 8]
 [0, 1, 2, 3, 4, 5, 6, 7, 8]

H-function Option 2 - Manhattan Distance Admissible

In [25]:
@time final_path = A_star(start2,goal, 2)

[1, 3, 4, 2, 7, 5, 6, 8, 0] 0.0 7.0 1 0
[1, 3, 4, 2, 7, 5, 6, 0, 8] 1.0 6.0 2 1
[1, 3, 4, 2, 0, 5, 6, 7, 8] 2.0 5.0 3 2
[1, 3, 4, 0, 2, 5, 6, 7, 8] 3.0 4.0 5 3
[1, 0, 4, 2, 3, 5, 6, 7, 8] 3.0 4.0 6 4
[0, 1, 4, 2, 3, 5, 6, 7, 8] 4.0 3.0 7 5
[0, 3, 4, 1, 2, 5, 6, 7, 8] 4.0 4.0 7 6
[1, 3, 4, 2, 7, 5, 0, 6, 8] 2.0 6.0 7 7
[2, 1, 4, 0, 3, 5, 6, 7, 8] 5.0 3.0 7 8
[1, 3, 4, 2, 7, 0, 6, 8, 5] 1.0 7.0 8 9
[1, 3, 0, 2, 7, 4, 6, 8, 5] 2.0 6.0 9 10
[1, 4, 0, 2, 3, 5, 6, 7, 8] 4.0 4.0 9 11
[1, 3, 4, 0, 7, 5, 2, 6, 8] 3.0 6.0 9 12
[1, 0, 3, 2, 7, 4, 6, 8, 5] 3.0 6.0 10 13
[0, 1, 3, 2, 7, 4, 6, 8, 5] 4.0 5.0 11 14
[1, 3, 4, 2, 0, 7, 6, 8, 5] 2.0 7.0 11 15
[1, 0, 4, 2, 3, 7, 6, 8, 5] 3.0 6.0 13 16
[0, 1, 4, 2, 3, 7, 6, 8, 5] 4.0 5.0 14 17
[1, 3, 4, 6, 2, 5, 0, 7, 8] 4.0 5.0 14 18
[1, 3, 4, 2, 5, 0, 6, 7, 8] 3.0 6.0 14 19
[1, 3, 4, 0, 2, 7, 6, 8, 5] 3.0 6.0 15 20
[1, 3, 0, 2, 5, 4, 6, 7, 8] 4.0 5.0 16 21
[2, 1, 4, 3, 0, 5, 6, 7, 8] 6.0 3.0 16 22
[3, 0, 4, 1, 2, 5, 6, 7, 8] 5.0 4.0 18 23
[1, 0, 3, 2, 5,

19-element Array{Any,1}:
 [1, 3, 4, 2, 7, 5, 6, 8, 0]
 [1, 3, 4, 2, 7, 5, 6, 0, 8]
 [1, 3, 4, 2, 0, 5, 6, 7, 8]
 [1, 0, 4, 2, 3, 5, 6, 7, 8]
 [1, 4, 0, 2, 3, 5, 6, 7, 8]
 [1, 4, 5, 2, 3, 0, 6, 7, 8]
 [1, 4, 5, 2, 0, 3, 6, 7, 8]
 [1, 4, 5, 0, 2, 3, 6, 7, 8]
 [0, 4, 5, 1, 2, 3, 6, 7, 8]
 [4, 0, 5, 1, 2, 3, 6, 7, 8]
 [4, 2, 5, 1, 0, 3, 6, 7, 8]
 [4, 2, 5, 1, 3, 0, 6, 7, 8]
 [4, 2, 0, 1, 3, 5, 6, 7, 8]
 [4, 0, 2, 1, 3, 5, 6, 7, 8]
 [0, 4, 2, 1, 3, 5, 6, 7, 8]
 [1, 4, 2, 0, 3, 5, 6, 7, 8]
 [1, 4, 2, 3, 0, 5, 6, 7, 8]
 [1, 0, 2, 3, 4, 5, 6, 7, 8]
 [0, 1, 2, 3, 4, 5, 6, 7, 8]

H-function Option 3 - Hamming Distance Weighted Non-Admissible

In [28]:
@time final_path = A_star(start2,goal, 3)

[1, 3, 4, 2, 7, 5, 6, 8, 0] 0.0 26.0 1 0
[1, 3, 4, 2, 7, 5, 6, 0, 8] 1.0 18.0 2 1
[1, 3, 4, 2, 0, 5, 6, 7, 8] 2.0 13.0 3 2
[1, 3, 4, 0, 2, 5, 6, 7, 8] 3.0 13.0 5 3
[1, 0, 4, 2, 3, 5, 6, 7, 8] 3.0 13.0 6 4
[0, 1, 4, 2, 3, 5, 6, 7, 8] 4.0 11.0 7 5
[0, 3, 4, 1, 2, 5, 6, 7, 8] 4.0 12.0 7 6
[2, 1, 4, 0, 3, 5, 6, 7, 8] 5.0 12.0 7 7
[2, 1, 4, 3, 0, 5, 6, 7, 8] 6.0 11.0 8 8
[1, 4, 0, 2, 3, 5, 6, 7, 8] 4.0 13.0 10 9
[3, 0, 4, 1, 2, 5, 6, 7, 8] 5.0 13.0 10 10
[3, 2, 4, 1, 0, 5, 6, 7, 8] 6.0 13.0 11 11
[2, 0, 4, 3, 1, 5, 6, 7, 8] 7.0 12.0 13 12
[0, 2, 4, 3, 1, 5, 6, 7, 8] 8.0 11.0 14 13
[1, 3, 4, 6, 2, 5, 0, 7, 8] 4.0 15.0 13 14
[3, 4, 0, 1, 2, 5, 6, 7, 8] 6.0 13.0 13 15
[3, 2, 4, 0, 1, 5, 6, 7, 8] 7.0 13.0 13 16
[2, 1, 4, 6, 3, 5, 0, 7, 8] 6.0 14.0 13 17
[2, 4, 0, 3, 1, 5, 6, 7, 8] 8.0 12.0 13 18
[1, 3, 4, 2, 5, 0, 6, 7, 8] 3.0 18.0 13 19
[1, 3, 4, 2, 7, 5, 0, 6, 8] 2.0 20.0 14 20
[1, 3, 0, 2, 5, 4, 6, 7, 8] 4.0 18.0 14 21
[1, 0, 3, 2, 5, 4, 6, 7, 8] 5.0 18.0 14 22
[0, 1, 3, 2, 5, 4, 6, 7, 8] 6.

19-element Array{Any,1}:
 [1, 3, 4, 2, 7, 5, 6, 8, 0]
 [1, 3, 4, 2, 7, 5, 6, 0, 8]
 [1, 3, 4, 2, 0, 5, 6, 7, 8]
 [1, 0, 4, 2, 3, 5, 6, 7, 8]
 [1, 4, 0, 2, 3, 5, 6, 7, 8]
 [1, 4, 5, 2, 3, 0, 6, 7, 8]
 [1, 4, 5, 2, 0, 3, 6, 7, 8]
 [1, 4, 5, 0, 2, 3, 6, 7, 8]
 [0, 4, 5, 1, 2, 3, 6, 7, 8]
 [4, 0, 5, 1, 2, 3, 6, 7, 8]
 [4, 2, 5, 1, 0, 3, 6, 7, 8]
 [4, 2, 5, 1, 3, 0, 6, 7, 8]
 [4, 2, 0, 1, 3, 5, 6, 7, 8]
 [4, 0, 2, 1, 3, 5, 6, 7, 8]
 [0, 4, 2, 1, 3, 5, 6, 7, 8]
 [1, 4, 2, 0, 3, 5, 6, 7, 8]
 [1, 4, 2, 3, 0, 5, 6, 7, 8]
 [1, 0, 2, 3, 4, 5, 6, 7, 8]
 [0, 1, 2, 3, 4, 5, 6, 7, 8]

H-function Option 4 - Manhattan Distance Alternative Non-Admissible

In [29]:
@time final_path = A_star(start2,goal, 4)

[1, 3, 4, 2, 7, 5, 6, 8, 0] 0.0 18.0 1 0
[1, 3, 4, 2, 7, 5, 6, 0, 8] 1.0 16.0 2 1
[1, 3, 4, 2, 0, 5, 6, 7, 8] 2.0 10.0 3 2
[1, 0, 4, 2, 3, 5, 6, 7, 8] 3.0 6.0 5 3
[0, 1, 4, 2, 3, 5, 6, 7, 8] 4.0 4.0 6 4
[1, 4, 0, 2, 3, 5, 6, 7, 8] 4.0 8.0 6 5
[1, 3, 4, 0, 2, 5, 6, 7, 8] 3.0 10.0 6 6
[0, 3, 4, 1, 2, 5, 6, 7, 8] 4.0 8.0 7 7
[2, 1, 4, 0, 3, 5, 6, 7, 8] 5.0 8.0 7 8
[2, 1, 4, 3, 0, 5, 6, 7, 8] 6.0 8.0 8 9
[2, 0, 4, 3, 1, 5, 6, 7, 8] 7.0 8.0 10 10
[0, 2, 4, 3, 1, 5, 6, 7, 8] 8.0 6.0 11 11
[1, 3, 4, 2, 5, 0, 6, 7, 8] 3.0 12.0 11 12
[1, 3, 0, 2, 5, 4, 6, 7, 8] 4.0 8.0 12 13
[1, 0, 3, 2, 5, 4, 6, 7, 8] 5.0 6.0 12 14
[0, 1, 3, 2, 5, 4, 6, 7, 8] 6.0 4.0 13 15
[2, 1, 3, 0, 5, 4, 6, 7, 8] 7.0 8.0 13 16
[3, 0, 4, 1, 2, 5, 6, 7, 8] 5.0 10.0 14 17
[2, 1, 4, 3, 5, 0, 6, 7, 8] 7.0 10.0 15 18
[2, 1, 0, 3, 5, 4, 6, 7, 8] 8.0 6.0 16 19
[2, 0, 1, 3, 5, 4, 6, 7, 8] 9.0 6.0 16 20
[0, 2, 1, 3, 5, 4, 6, 7, 8] 10.0 4.0 17 21
[3, 2, 4, 1, 0, 5, 6, 7, 8] 6.0 12.0 17 22
[1, 3, 4, 2, 7, 5, 0, 6, 8] 2.0 16.0 18 23
[1

19-element Array{Any,1}:
 [1, 3, 4, 2, 7, 5, 6, 8, 0]
 [1, 3, 4, 2, 7, 5, 6, 0, 8]
 [1, 3, 4, 2, 0, 5, 6, 7, 8]
 [1, 0, 4, 2, 3, 5, 6, 7, 8]
 [1, 4, 0, 2, 3, 5, 6, 7, 8]
 [1, 4, 5, 2, 3, 0, 6, 7, 8]
 [1, 4, 5, 2, 0, 3, 6, 7, 8]
 [1, 4, 5, 0, 2, 3, 6, 7, 8]
 [0, 4, 5, 1, 2, 3, 6, 7, 8]
 [4, 0, 5, 1, 2, 3, 6, 7, 8]
 [4, 2, 5, 1, 0, 3, 6, 7, 8]
 [4, 2, 5, 1, 3, 0, 6, 7, 8]
 [4, 2, 0, 1, 3, 5, 6, 7, 8]
 [4, 0, 2, 1, 3, 5, 6, 7, 8]
 [0, 4, 2, 1, 3, 5, 6, 7, 8]
 [1, 4, 2, 0, 3, 5, 6, 7, 8]
 [1, 4, 2, 3, 0, 5, 6, 7, 8]
 [1, 0, 2, 3, 4, 5, 6, 7, 8]
 [0, 1, 2, 3, 4, 5, 6, 7, 8]

H-function Option 5 - Weighted Manhattan Distance Non-Admissible

In [53]:
@time final_path = A_star(start2,goal, 5)

[1, 3, 4, 2, 7, 5, 6, 8, 0] 0.0 15.25 1 0
[1, 3, 4, 2, 7, 5, 6, 0, 8] 1.0 10.25 2 1
[1, 3, 4, 2, 0, 5, 6, 7, 8] 2.0 6.75 3 2
[1, 0, 4, 2, 3, 5, 6, 7, 8] 3.0 5.25 5 3
[0, 1, 4, 2, 3, 5, 6, 7, 8] 4.0 4.5 6 4
[1, 3, 4, 0, 2, 5, 6, 7, 8] 3.0 5.75 6 5
[2, 1, 4, 0, 3, 5, 6, 7, 8] 5.0 4.0 7 6
[1, 4, 0, 2, 3, 5, 6, 7, 8] 4.0 5.25 8 7
[0, 3, 4, 1, 2, 5, 6, 7, 8] 4.0 6.0 8 8
[2, 1, 4, 3, 0, 5, 6, 7, 8] 6.0 4.5 8 9
[3, 0, 4, 1, 2, 5, 6, 7, 8] 5.0 5.75 10 10
[2, 0, 4, 3, 1, 5, 6, 7, 8] 7.0 4.0 11 11
[1, 3, 4, 2, 7, 5, 0, 6, 8] 2.0 9.25 12 12
[0, 2, 4, 3, 1, 5, 6, 7, 8] 8.0 3.5 12 13
[3, 2, 4, 1, 0, 5, 6, 7, 8] 6.0 5.75 12 14
[3, 2, 4, 0, 1, 5, 6, 7, 8] 7.0 4.25 13 15
[1, 3, 4, 6, 2, 5, 0, 7, 8] 4.0 7.75 13 16
[3, 4, 0, 1, 2, 5, 6, 7, 8] 6.0 5.75 13 17
[2, 1, 4, 6, 3, 5, 0, 7, 8] 6.0 6.0 13 18
[2, 4, 0, 3, 1, 5, 6, 7, 8] 8.0 4.0 13 19
[1, 3, 4, 0, 7, 5, 2, 6, 8] 3.0 10.25 13 20
[1, 3, 4, 2, 5, 0, 6, 7, 8] 3.0 10.25 14 21
[1, 3, 0, 2, 5, 4, 6, 7, 8] 4.0 7.25 15 22
[1, 0, 3, 2, 5, 4, 6, 7, 8] 5.0 7.7

19-element Array{Any,1}:
 [1, 3, 4, 2, 7, 5, 6, 8, 0]
 [1, 3, 4, 2, 7, 5, 6, 0, 8]
 [1, 3, 4, 2, 0, 5, 6, 7, 8]
 [1, 0, 4, 2, 3, 5, 6, 7, 8]
 [1, 4, 0, 2, 3, 5, 6, 7, 8]
 [1, 4, 5, 2, 3, 0, 6, 7, 8]
 [1, 4, 5, 2, 0, 3, 6, 7, 8]
 [1, 4, 5, 0, 2, 3, 6, 7, 8]
 [0, 4, 5, 1, 2, 3, 6, 7, 8]
 [4, 0, 5, 1, 2, 3, 6, 7, 8]
 [4, 2, 5, 1, 0, 3, 6, 7, 8]
 [4, 2, 5, 1, 3, 0, 6, 7, 8]
 [4, 2, 0, 1, 3, 5, 6, 7, 8]
 [4, 0, 2, 1, 3, 5, 6, 7, 8]
 [0, 4, 2, 1, 3, 5, 6, 7, 8]
 [1, 4, 2, 0, 3, 5, 6, 7, 8]
 [1, 4, 2, 3, 0, 5, 6, 7, 8]
 [1, 0, 2, 3, 4, 5, 6, 7, 8]
 [0, 1, 2, 3, 4, 5, 6, 7, 8]

### 2.4.3. Difficult Example
The following is a quiet complicated example.

In [30]:
start3 = [8,1,7,4,5,6,2,0,3]

9-element Array{Int64,1}:
 8
 1
 7
 4
 5
 6
 2
 0
 3

H-function Option 1 - Hamming Distance Admissible

In [31]:
@time final_path = A_star(start3,goal, 1, 100)

[8, 1, 7, 4, 5, 6, 2, 0, 3] 0.0 8.0 1 0
[8, 1, 7, 2, 0, 4, 5, 3, 6] 7.0 8.0 71 100
[1, 0, 7, 8, 4, 3, 2, 6, 5] 8.0 8.0 126 200
[0, 8, 6, 5, 7, 1, 4, 2, 3] 9.0 8.0 181 300
[8, 5, 1, 4, 0, 6, 2, 7, 3] 9.0 8.0 224 400
[4, 8, 7, 1, 6, 3, 0, 2, 5] 9.0 9.0 297 500
[4, 6, 8, 1, 0, 7, 2, 5, 3] 9.0 9.0 355 600
[8, 0, 1, 4, 5, 3, 2, 6, 7] 10.0 9.0 417 700
[8, 1, 7, 4, 3, 0, 6, 5, 2] 12.0 7.0 471 800
[8, 0, 7, 4, 5, 6, 2, 3, 1] 10.0 9.0 539 900
[8, 1, 7, 3, 6, 0, 4, 5, 2] 12.0 7.0 598 1000
[4, 8, 7, 2, 3, 1, 5, 0, 6] 10.0 9.0 645 1100
[2, 1, 7, 0, 5, 6, 8, 4, 3] 12.0 8.0 697 1200
[0, 4, 1, 3, 8, 7, 2, 6, 5] 13.0 7.0 758 1300
[2, 8, 7, 1, 4, 5, 0, 3, 6] 13.0 7.0 818 1400
[8, 1, 4, 2, 6, 7, 5, 0, 3] 12.0 8.0 867 1500
[1, 4, 0, 2, 8, 7, 5, 3, 6] 11.0 9.0 907 1600
[2, 8, 1, 6, 4, 7, 5, 0, 3] 12.0 8.0 952 1700
[4, 8, 3, 0, 7, 1, 2, 6, 5] 12.0 9.0 998 1800
[1, 6, 0, 8, 7, 5, 2, 3, 4] 13.0 8.0 1065 1900
[4, 0, 8, 6, 5, 1, 2, 3, 7] 12.0 9.0 1119 2000
[5, 8, 1, 0, 3, 7, 4, 6, 2] 12.0 9.0 1170 2100
[1, 0, 

26-element Array{Any,1}:
 [8, 1, 7, 4, 5, 6, 2, 0, 3]
 [8, 1, 7, 4, 5, 6, 0, 2, 3]
 [8, 1, 7, 0, 5, 6, 4, 2, 3]
 [0, 1, 7, 8, 5, 6, 4, 2, 3]
 [1, 0, 7, 8, 5, 6, 4, 2, 3]
 [1, 5, 7, 8, 0, 6, 4, 2, 3]
 [1, 5, 7, 8, 2, 6, 4, 0, 3]
 [1, 5, 7, 8, 2, 6, 4, 3, 0]
 [1, 5, 7, 8, 2, 0, 4, 3, 6]
 [1, 5, 0, 8, 2, 7, 4, 3, 6]
 [1, 0, 5, 8, 2, 7, 4, 3, 6]
 [1, 2, 5, 8, 0, 7, 4, 3, 6]
 [1, 2, 5, 0, 8, 7, 4, 3, 6]
 [1, 2, 5, 4, 8, 7, 0, 3, 6]
 [1, 2, 5, 4, 8, 7, 3, 0, 6]
 [1, 2, 5, 4, 8, 7, 3, 6, 0]
 [1, 2, 5, 4, 8, 0, 3, 6, 7]
 [1, 2, 5, 4, 0, 8, 3, 6, 7]
 [1, 2, 5, 0, 4, 8, 3, 6, 7]
 [1, 2, 5, 3, 4, 8, 0, 6, 7]
 [1, 2, 5, 3, 4, 8, 6, 0, 7]
 [1, 2, 5, 3, 4, 8, 6, 7, 0]
 [1, 2, 5, 3, 4, 0, 6, 7, 8]
 [1, 2, 0, 3, 4, 5, 6, 7, 8]
 [1, 0, 2, 3, 4, 5, 6, 7, 8]
 [0, 1, 2, 3, 4, 5, 6, 7, 8]

H-function Option 2 - Manhattan Distance Admissible

In [32]:
@time final_path = A_star(start3,goal, 2, 100)

[8, 1, 7, 4, 5, 6, 2, 0, 3] 0.0 11.0 1 0
[8, 7, 0, 4, 1, 3, 2, 6, 5] 7.0 9.0 66 100
[0, 8, 1, 2, 4, 7, 5, 3, 6] 9.0 8.0 126 200
[1, 4, 7, 8, 5, 6, 2, 3, 0] 7.0 11.0 182 300
[0, 8, 1, 4, 6, 5, 2, 3, 7] 11.0 7.0 232 400
[3, 8, 7, 4, 1, 5, 0, 2, 6] 11.0 8.0 292 500
[4, 8, 1, 5, 3, 7, 2, 6, 0] 9.0 10.0 356 600
[2, 8, 7, 0, 1, 3, 6, 4, 5] 12.0 7.0 406 700
[4, 1, 0, 6, 8, 5, 2, 3, 7] 13.0 7.0 453 800
[1, 2, 0, 8, 6, 7, 5, 4, 3] 11.0 9.0 519 900
[8, 7, 0, 3, 1, 2, 4, 6, 5] 13.0 7.0 578 1000
[8, 2, 1, 5, 3, 0, 4, 6, 7] 12.0 8.0 631 1100
[1, 2, 7, 0, 4, 6, 8, 5, 3] 12.0 8.0 688 1200
[1, 4, 7, 5, 2, 6, 0, 8, 3] 11.0 9.0 741 1300
[4, 6, 0, 1, 7, 8, 2, 5, 3] 11.0 10.0 788 1400
[2, 8, 7, 1, 4, 5, 3, 6, 0] 13.0 8.0 834 1500
[4, 8, 0, 2, 3, 7, 6, 1, 5] 13.0 8.0 892 1600
[1, 2, 7, 8, 3, 4, 5, 0, 6] 12.0 9.0 953 1700
[8, 0, 7, 5, 1, 4, 3, 2, 6] 12.0 9.0 1013 1800
[0, 1, 5, 8, 2, 7, 6, 4, 3] 15.0 6.0 1065 1900
[3, 4, 1, 0, 8, 7, 2, 6, 5] 14.0 7.0 1119 2000
[8, 0, 7, 6, 1, 2, 4, 5, 3] 12.0 9.0 1173 2100


26-element Array{Any,1}:
 [8, 1, 7, 4, 5, 6, 2, 0, 3]
 [8, 1, 7, 4, 5, 6, 0, 2, 3]
 [8, 1, 7, 0, 5, 6, 4, 2, 3]
 [0, 1, 7, 8, 5, 6, 4, 2, 3]
 [1, 0, 7, 8, 5, 6, 4, 2, 3]
 [1, 5, 7, 8, 0, 6, 4, 2, 3]
 [1, 5, 7, 8, 2, 6, 4, 0, 3]
 [1, 5, 7, 8, 2, 6, 4, 3, 0]
 [1, 5, 7, 8, 2, 0, 4, 3, 6]
 [1, 5, 0, 8, 2, 7, 4, 3, 6]
 [1, 0, 5, 8, 2, 7, 4, 3, 6]
 [1, 2, 5, 8, 0, 7, 4, 3, 6]
 [1, 2, 5, 0, 8, 7, 4, 3, 6]
 [1, 2, 5, 4, 8, 7, 0, 3, 6]
 [1, 2, 5, 4, 8, 7, 3, 0, 6]
 [1, 2, 5, 4, 8, 7, 3, 6, 0]
 [1, 2, 5, 4, 8, 0, 3, 6, 7]
 [1, 2, 5, 4, 0, 8, 3, 6, 7]
 [1, 2, 5, 0, 4, 8, 3, 6, 7]
 [1, 2, 5, 3, 4, 8, 0, 6, 7]
 [1, 2, 5, 3, 4, 8, 6, 0, 7]
 [1, 2, 5, 3, 4, 8, 6, 7, 0]
 [1, 2, 5, 3, 4, 0, 6, 7, 8]
 [1, 2, 0, 3, 4, 5, 6, 7, 8]
 [1, 0, 2, 3, 4, 5, 6, 7, 8]
 [0, 1, 2, 3, 4, 5, 6, 7, 8]

H-function Option 3 - Hamming Distance Weighted Non-Admissible

In [35]:
@time final_path = A_star(start3,goal, 3, 100)

[8, 1, 7, 4, 5, 6, 2, 0, 3] 0.0 32.0 1 0
[8, 1, 7, 5, 2, 0, 4, 3, 6] 6.0 32.0 57 100
[0, 8, 1, 5, 6, 7, 4, 2, 3] 7.0 32.0 129 200
[8, 7, 6, 2, 4, 1, 5, 3, 0] 9.0 31.0 192 300
[0, 8, 7, 2, 1, 5, 6, 3, 4] 15.0 25.0 226 400
[4, 8, 7, 0, 2, 6, 5, 1, 3] 8.0 33.0 286 500
[8, 5, 1, 3, 0, 7, 4, 2, 6] 9.0 32.0 348 600
[8, 1, 7, 6, 4, 3, 5, 0, 2] 10.0 30.0 402 700
[4, 0, 6, 7, 8, 5, 2, 3, 1] 14.0 28.0 455 800
[8, 1, 6, 0, 7, 4, 2, 5, 3] 10.0 32.0 521 900
[2, 1, 7, 4, 0, 5, 3, 8, 6] 15.0 27.0 579 1000
[1, 5, 0, 8, 2, 7, 4, 3, 6] 9.0 33.0 634 1100
[0, 8, 1, 2, 3, 7, 6, 4, 5] 13.0 30.0 676 1200
[8, 6, 3, 0, 4, 5, 2, 7, 1] 14.0 21.0 745 1300
[3, 1, 0, 8, 4, 5, 2, 7, 6] 17.0 20.0 810 1400
[1, 7, 3, 8, 6, 0, 4, 2, 5] 10.0 33.0 864 1500
[0, 1, 5, 8, 2, 7, 4, 3, 6] 11.0 31.0 924 1600
[8, 2, 7, 4, 1, 5, 0, 3, 6] 15.0 28.0 973 1700
[8, 7, 5, 2, 4, 0, 3, 6, 1] 12.0 31.0 1022 1800
[0, 7, 6, 1, 8, 3, 2, 4, 5] 11.0 32.0 1076 1900
[8, 1, 6, 2, 7, 4, 0, 5, 3] 11.0 32.0 1129 2000
[8, 1, 0, 3, 6, 7, 4, 5, 2] 13.0

34-element Array{Any,1}:
 [8, 1, 7, 4, 5, 6, 2, 0, 3]
 [8, 1, 7, 4, 5, 6, 0, 2, 3]
 [8, 1, 7, 0, 5, 6, 4, 2, 3]
 [0, 1, 7, 8, 5, 6, 4, 2, 3]
 [1, 0, 7, 8, 5, 6, 4, 2, 3]
 [1, 5, 7, 8, 0, 6, 4, 2, 3]
 [1, 5, 7, 8, 6, 0, 4, 2, 3]
 [1, 5, 7, 8, 6, 3, 4, 2, 0]
 [1, 5, 7, 8, 6, 3, 4, 0, 2]
 [1, 5, 7, 8, 0, 3, 4, 6, 2]
 [1, 5, 7, 0, 8, 3, 4, 6, 2]
 [1, 5, 7, 4, 8, 3, 0, 6, 2]
 [1, 5, 7, 4, 8, 3, 6, 0, 2]
 ⋮                          
 [1, 7, 2, 4, 3, 5, 6, 0, 8]
 [1, 7, 2, 4, 3, 5, 0, 6, 8]
 [1, 7, 2, 0, 3, 5, 4, 6, 8]
 [1, 7, 2, 3, 0, 5, 4, 6, 8]
 [1, 0, 2, 3, 7, 5, 4, 6, 8]
 [0, 1, 2, 3, 7, 5, 4, 6, 8]
 [3, 1, 2, 0, 7, 5, 4, 6, 8]
 [3, 1, 2, 4, 7, 5, 0, 6, 8]
 [3, 1, 2, 4, 7, 5, 6, 0, 8]
 [3, 1, 2, 4, 0, 5, 6, 7, 8]
 [3, 1, 2, 0, 4, 5, 6, 7, 8]
 [0, 1, 2, 3, 4, 5, 6, 7, 8]

H-function Option 4 - Manhattan Distance Alternative Non-Admissible

In [36]:
@time final_path = A_star(start3,goal, 4, 100)

[8, 1, 7, 4, 5, 6, 2, 0, 3] 0.0 32.0 1 0
[1, 2, 7, 8, 0, 6, 5, 4, 3] 9.0 26.0 64 100
[2, 1, 4, 0, 5, 7, 3, 8, 6] 16.0 16.0 130 200
[2, 1, 4, 0, 6, 7, 5, 8, 3] 14.0 18.0 203 300
[1, 0, 7, 2, 3, 4, 5, 8, 6] 22.0 14.0 271 400
[8, 1, 7, 5, 0, 2, 4, 3, 6] 7.0 30.0 333 500
[8, 7, 0, 2, 1, 4, 5, 3, 6] 9.0 28.0 397 600
[0, 2, 8, 4, 3, 1, 5, 6, 7] 17.0 16.0 452 700
[8, 1, 3, 0, 4, 5, 2, 6, 7] 14.0 18.0 507 800
[2, 4, 1, 3, 7, 6, 5, 0, 8] 18.0 18.0 554 900
[1, 7, 0, 8, 6, 3, 4, 2, 5] 9.0 28.0 615 1000
[8, 0, 1, 3, 2, 5, 6, 4, 7] 20.0 16.0 676 1100
[1, 0, 3, 5, 2, 4, 6, 8, 7] 22.0 10.0 734 1200
[0, 3, 7, 1, 2, 6, 5, 4, 8] 21.0 16.0 798 1300
[0, 1, 7, 3, 2, 5, 6, 4, 8] 23.0 10.0 860 1400
[8, 1, 4, 0, 5, 7, 2, 3, 6] 12.0 26.0 924 1500
[2, 0, 3, 1, 4, 7, 6, 8, 5] 22.0 12.0 991 1600
[1, 4, 0, 2, 3, 7, 6, 8, 5] 23.0 14.0 1054 1700
[1, 0, 3, 2, 4, 7, 5, 8, 6] 24.0 10.0 1107 1800
[2, 1, 3, 4, 5, 0, 6, 8, 7] 26.0 12.0 1172 1900
[2, 1, 8, 3, 4, 5, 6, 7, 0] 23.0 16.0 1219 2000
[2, 0, 4, 8, 5, 1, 3, 6, 7] 1

28-element Array{Any,1}:
 [8, 1, 7, 4, 5, 6, 2, 0, 3]
 [8, 1, 7, 4, 5, 6, 2, 3, 0]
 [8, 1, 7, 4, 5, 0, 2, 3, 6]
 [8, 1, 0, 4, 5, 7, 2, 3, 6]
 [8, 0, 1, 4, 5, 7, 2, 3, 6]
 [0, 8, 1, 4, 5, 7, 2, 3, 6]
 [4, 8, 1, 0, 5, 7, 2, 3, 6]
 [4, 8, 1, 2, 5, 7, 0, 3, 6]
 [4, 8, 1, 2, 5, 7, 3, 0, 6]
 [4, 8, 1, 2, 5, 7, 3, 6, 0]
 [4, 8, 1, 2, 5, 0, 3, 6, 7]
 [4, 8, 1, 2, 0, 5, 3, 6, 7]
 [4, 0, 1, 2, 8, 5, 3, 6, 7]
 ⋮                          
 [4, 1, 5, 0, 2, 8, 3, 6, 7]
 [0, 1, 5, 4, 2, 8, 3, 6, 7]
 [1, 0, 5, 4, 2, 8, 3, 6, 7]
 [1, 2, 5, 4, 0, 8, 3, 6, 7]
 [1, 2, 5, 0, 4, 8, 3, 6, 7]
 [1, 2, 5, 3, 4, 8, 0, 6, 7]
 [1, 2, 5, 3, 4, 8, 6, 0, 7]
 [1, 2, 5, 3, 4, 8, 6, 7, 0]
 [1, 2, 5, 3, 4, 0, 6, 7, 8]
 [1, 2, 0, 3, 4, 5, 6, 7, 8]
 [1, 0, 2, 3, 4, 5, 6, 7, 8]
 [0, 1, 2, 3, 4, 5, 6, 7, 8]

H-function Option 5 - Manhattan Distance Weighted Non-Admissible

In [54]:
@time final_path = A_star(start3,goal, 5, 100)

[8, 1, 7, 4, 5, 6, 2, 0, 3] 0.0 24.5 1 0
[1, 0, 7, 8, 2, 4, 3, 6, 5] 12.0 13.25 69 100
[1, 0, 7, 8, 4, 6, 2, 5, 3] 4.0 22.75 128 200
[1, 3, 7, 8, 0, 5, 4, 2, 6] 9.0 18.25 194 300
[8, 7, 5, 4, 3, 1, 2, 0, 6] 8.0 20.0 246 400
[8, 2, 1, 4, 0, 5, 3, 6, 7] 15.0 9.5 309 500
[4, 0, 1, 6, 8, 7, 2, 5, 3] 8.0 20.5 377 600
[1, 0, 4, 8, 3, 7, 2, 6, 5] 12.0 15.75 430 700
[8, 1, 7, 6, 3, 5, 0, 4, 2] 11.0 13.0 485 800
[8, 1, 0, 3, 4, 7, 6, 2, 5] 15.0 12.5 532 900
[5, 2, 0, 8, 4, 1, 3, 6, 7] 17.0 12.25 590 1000
[8, 1, 7, 4, 0, 3, 6, 5, 2] 11.0 16.5 654 1100
[0, 1, 7, 8, 5, 4, 3, 2, 6] 13.0 16.5 710 1200
[1, 6, 7, 0, 2, 3, 8, 4, 5] 12.0 15.75 770 1300
[8, 0, 7, 2, 1, 5, 6, 3, 4] 16.0 14.0 853 1400
[6, 0, 1, 8, 2, 4, 3, 7, 5] 18.0 10.0 907 1500
[0, 2, 8, 3, 4, 1, 6, 5, 7] 19.0 10.5 959 1600
[8, 3, 2, 0, 5, 1, 4, 6, 7] 16.0 12.0 1015 1700
[8, 5, 1, 2, 6, 0, 3, 7, 4] 14.0 16.0 1068 1800
[0, 4, 7, 1, 3, 2, 8, 6, 5] 17.0 12.5 1134 1900
[1, 0, 6, 8, 7, 4, 2, 5, 3] 8.0 21.75 1199 2000
[6, 0, 1, 8, 3, 2, 4, 7,

30-element Array{Any,1}:
 [8, 1, 7, 4, 5, 6, 2, 0, 3]
 [8, 1, 7, 4, 5, 6, 2, 3, 0]
 [8, 1, 7, 4, 5, 0, 2, 3, 6]
 [8, 1, 7, 4, 0, 5, 2, 3, 6]
 [8, 1, 7, 0, 4, 5, 2, 3, 6]
 [8, 1, 7, 2, 4, 5, 0, 3, 6]
 [8, 1, 7, 2, 4, 5, 3, 0, 6]
 [8, 1, 7, 2, 0, 5, 3, 4, 6]
 [8, 0, 7, 2, 1, 5, 3, 4, 6]
 [0, 8, 7, 2, 1, 5, 3, 4, 6]
 [2, 8, 7, 0, 1, 5, 3, 4, 6]
 [2, 8, 7, 1, 0, 5, 3, 4, 6]
 [2, 0, 7, 1, 8, 5, 3, 4, 6]
 ⋮                          
 [2, 7, 5, 1, 4, 0, 3, 6, 8]
 [2, 7, 5, 1, 0, 4, 3, 6, 8]
 [2, 0, 5, 1, 7, 4, 3, 6, 8]
 [0, 2, 5, 1, 7, 4, 3, 6, 8]
 [1, 2, 5, 0, 7, 4, 3, 6, 8]
 [1, 2, 5, 3, 7, 4, 0, 6, 8]
 [1, 2, 5, 3, 7, 4, 6, 0, 8]
 [1, 2, 5, 3, 0, 4, 6, 7, 8]
 [1, 2, 5, 3, 4, 0, 6, 7, 8]
 [1, 2, 0, 3, 4, 5, 6, 7, 8]
 [1, 0, 2, 3, 4, 5, 6, 7, 8]
 [0, 1, 2, 3, 4, 5, 6, 7, 8]

### 2.4.4. Unsolvable Example
The next example is unsolvable, as can be seen by parity of permutation.

In [37]:
start4 = [4,6,3,5,7,1,2,8,0]

9-element Array{Int64,1}:
 4
 6
 3
 5
 7
 1
 2
 8
 0

H-function Option 1 - Hamming Distance Admissible

In [38]:
@time final_path = A_star(start4,goal, 1, 1000)

[4, 6, 3, 5, 7, 1, 2, 8, 0] 0.0 9.0 1 0
[4, 8, 6, 2, 5, 3, 0, 1, 7] 10.0 9.0 585 1000
[0, 5, 1, 3, 7, 6, 4, 2, 8] 14.0 6.0 1158 2000
[1, 5, 3, 0, 6, 4, 2, 7, 8] 15.0 7.0 1699 3000
[5, 4, 6, 0, 2, 1, 7, 8, 3] 13.0 9.0 2296 4000
[1, 5, 6, 7, 2, 3, 4, 0, 8] 15.0 8.0 2785 5000
[0, 4, 2, 5, 7, 3, 8, 6, 1] 16.0 7.0 3320 6000
[6, 1, 5, 3, 0, 8, 4, 2, 7] 16.0 7.0 3769 7000
[1, 4, 3, 2, 0, 8, 6, 5, 7] 16.0 8.0 4298 8000
[5, 6, 4, 3, 7, 1, 2, 8, 0] 16.0 8.0 4841 9000
[2, 4, 3, 7, 6, 8, 1, 0, 5] 15.0 9.0 5352 10000
[5, 1, 8, 2, 4, 7, 6, 3, 0] 18.0 6.0 5753 11000
[4, 6, 2, 0, 8, 7, 5, 1, 3] 17.0 8.0 6121 12000
[6, 5, 4, 0, 3, 1, 7, 2, 8] 17.0 8.0 6587 13000
[6, 5, 7, 2, 4, 0, 8, 3, 1] 17.0 8.0 7029 14000
[7, 1, 6, 0, 4, 2, 5, 3, 8] 19.0 6.0 7433 15000
[0, 1, 6, 4, 5, 3, 8, 2, 7] 18.0 7.0 7831 16000
[0, 7, 4, 6, 3, 5, 2, 8, 1] 18.0 7.0 8177 17000
[7, 5, 4, 0, 1, 6, 8, 2, 3] 17.0 9.0 8552 18000
[6, 4, 3, 5, 0, 7, 2, 1, 8] 18.0 8.0 9088 19000
[4, 1, 0, 8, 6, 7, 5, 3, 2] 18.0 8.0 9575 20000
[4, 6, 7, 

"There exists no path."

H-function Option 2 - Manhattan Distance Admissible

In [39]:
@time final_path = A_star(start4,goal, 2, 1000)

[4, 6, 3, 5, 7, 1, 2, 8, 0] 0.0 11.0 1 0
[6, 3, 0, 4, 7, 1, 2, 8, 5] 12.0 8.0 569 1000
[4, 0, 3, 2, 5, 1, 7, 8, 6] 13.0 8.0 1144 2000
[0, 5, 4, 6, 2, 3, 7, 1, 8] 16.0 6.0 1700 3000
[4, 0, 2, 6, 3, 1, 7, 5, 8] 17.0 5.0 2226 4000
[5, 6, 0, 2, 4, 1, 3, 7, 8] 16.0 7.0 2758 5000
[4, 2, 3, 6, 0, 1, 5, 7, 8] 16.0 7.0 3292 6000
[6, 2, 3, 7, 4, 1, 5, 8, 0] 14.0 9.0 3723 7000
[5, 1, 0, 3, 2, 6, 7, 4, 8] 18.0 6.0 4265 8000
[6, 5, 3, 1, 0, 8, 7, 4, 2] 16.0 8.0 4783 9000
[0, 1, 6, 7, 4, 8, 5, 2, 3] 16.0 8.0 5236 10000
[3, 7, 5, 6, 8, 4, 2, 0, 1] 15.0 9.0 5643 11000
[3, 2, 5, 4, 6, 1, 7, 0, 8] 19.0 6.0 6100 12000
[6, 3, 7, 2, 4, 0, 5, 8, 1] 15.0 10.0 6599 13000
[3, 0, 5, 4, 8, 6, 2, 7, 1] 17.0 8.0 7063 14000
[0, 2, 7, 1, 6, 3, 4, 5, 8] 18.0 7.0 7469 15000
[0, 8, 7, 3, 4, 1, 5, 6, 2] 18.0 7.0 7891 16000
[3, 7, 6, 4, 8, 1, 5, 0, 2] 15.0 10.0 8241 17000
[4, 5, 3, 2, 7, 0, 8, 1, 6] 15.0 10.0 8585 18000
[0, 5, 6, 4, 7, 3, 2, 1, 8] 18.0 8.0 9040 19000
[6, 2, 0, 5, 8, 3, 4, 1, 7] 18.0 8.0 9465 20000
[6, 2,

"There exists no path."

H-function Option 3 - Hamming Distance Weighted Non-Admissible

In [42]:
@time final_path = A_star(start4,goal, 3, 1000)

[4, 6, 3, 5, 7, 1, 2, 8, 0] 0.0 33.0 1 0
[2, 3, 1, 0, 6, 5, 4, 7, 8] 19.0 15.0 574 1000
[0, 4, 2, 7, 1, 3, 6, 5, 8] 18.0 20.0 1119 2000
[7, 1, 5, 4, 0, 3, 6, 2, 8] 18.0 22.0 1701 3000
[3, 7, 0, 5, 6, 1, 2, 4, 8] 16.0 25.0 2278 4000
[6, 1, 8, 4, 3, 2, 5, 7, 0] 14.0 27.0 2753 5000
[4, 6, 2, 3, 1, 0, 5, 7, 8] 25.0 17.0 3194 6000
[7, 3, 1, 0, 4, 2, 5, 6, 8] 19.0 23.0 3652 7000
[6, 0, 3, 4, 8, 5, 1, 2, 7] 15.0 28.0 4025 8000
[6, 4, 0, 1, 3, 2, 5, 7, 8] 24.0 20.0 4478 9000
[3, 7, 0, 2, 1, 5, 4, 6, 8] 24.0 20.0 4952 10000
[2, 1, 6, 7, 0, 4, 5, 3, 8] 20.0 24.0 5307 11000
[4, 8, 3, 5, 6, 0, 2, 7, 1] 17.0 28.0 5658 12000
[0, 8, 1, 3, 4, 6, 5, 7, 2] 20.0 24.0 5998 13000
[2, 0, 7, 1, 6, 5, 3, 4, 8] 25.0 20.0 6267 14000
[4, 2, 3, 6, 1, 8, 5, 7, 0] 18.0 28.0 6644 15000
[5, 6, 3, 7, 8, 0, 2, 1, 4] 13.0 33.0 7086 16000
[4, 8, 6, 2, 3, 0, 5, 7, 1] 17.0 28.0 7473 17000
[0, 5, 8, 3, 4, 6, 2, 1, 7] 16.0 29.0 7818 18000
[6, 5, 3, 2, 1, 8, 7, 0, 4] 13.0 33.0 8158 19000
[1, 4, 0, 5, 6, 7, 3, 2, 8] 22.0 25.0 

"There exists no path."

H-function Option 4 - Manhattan Distance Alternative Non-Admissible

In [43]:
@time final_path = A_star(start4,goal, 4, 1000)

[4, 6, 3, 5, 7, 1, 2, 8, 0] 0.0 32.0 1 0
[3, 5, 1, 2, 4, 0, 7, 8, 6] 15.0 18.0 595 1000
[4, 0, 1, 5, 2, 3, 6, 7, 8] 19.0 12.0 1172 2000
[0, 2, 1, 6, 5, 4, 7, 8, 3] 20.0 14.0 1700 3000
[1, 5, 3, 2, 6, 4, 7, 0, 8] 19.0 18.0 2256 4000
[1, 0, 2, 3, 5, 7, 4, 6, 8] 27.0 8.0 2720 5000
[0, 2, 6, 3, 4, 1, 5, 7, 8] 26.0 10.0 3172 6000
[2, 0, 3, 7, 1, 4, 5, 8, 6] 23.0 16.0 3693 7000
[0, 2, 5, 4, 6, 3, 7, 8, 1] 18.0 18.0 4166 8000
[1, 5, 6, 7, 0, 3, 4, 2, 8] 14.0 26.0 4624 9000
[1, 4, 0, 7, 2, 6, 5, 3, 8] 22.0 18.0 4981 10000
[2, 5, 3, 1, 4, 6, 7, 8, 0] 20.0 20.0 5287 11000
[7, 1, 3, 0, 4, 2, 5, 8, 6] 21.0 18.0 5686 12000
[1, 4, 0, 2, 7, 3, 6, 8, 5] 22.0 16.0 6138 13000
[1, 0, 4, 6, 8, 3, 2, 7, 5] 19.0 20.0 6613 14000
[3, 0, 7, 5, 4, 1, 2, 8, 6] 17.0 22.0 7052 15000
[2, 3, 1, 0, 6, 5, 8, 4, 7] 25.0 16.0 7372 16000
[2, 4, 3, 8, 6, 1, 0, 7, 5] 16.0 26.0 7664 17000
[3, 0, 1, 7, 2, 4, 8, 6, 5] 23.0 18.0 7959 18000
[4, 6, 3, 7, 0, 1, 2, 8, 5] 12.0 30.0 8209 19000
[5, 3, 0, 6, 2, 4, 1, 7, 8] 20.0 20.0 8

"There exists no path."

H-function Option 5 - Manhattan Distance Weighted Non-Admissible

In [56]:
@time final_path = A_star(start4,goal, 5, 1000)

[4, 6, 3, 5, 7, 1, 2, 8, 0] 0.0 39.25 1 0
[2, 4, 5, 1, 0, 3, 6, 7, 8] 22.0 12.0 577 1000
[0, 5, 1, 6, 7, 4, 3, 2, 8] 18.0 17.0 1069 2000
[3, 5, 1, 6, 2, 8, 0, 4, 7] 18.0 18.75 1574 3000
[3, 2, 1, 7, 6, 4, 0, 8, 5] 20.0 18.75 1985 4000
[6, 2, 5, 3, 1, 0, 7, 4, 8] 23.0 16.5 2427 5000
[2, 0, 8, 3, 1, 4, 6, 5, 7] 21.0 17.75 2839 6000
[4, 1, 0, 6, 7, 5, 2, 3, 8] 24.0 17.0 3247 7000
[7, 1, 2, 3, 0, 5, 4, 6, 8] 30.0 9.5 3624 8000
[3, 0, 4, 7, 1, 5, 2, 6, 8] 27.0 15.0 4013 9000
[6, 1, 2, 4, 7, 3, 0, 8, 5] 22.0 17.75 4388 10000
[0, 3, 5, 2, 7, 6, 1, 4, 8] 22.0 21.0 4779 11000
[0, 1, 2, 3, 8, 4, 7, 6, 5] 26.0 14.0 5291 12000
[2, 4, 7, 1, 6, 3, 0, 5, 8] 20.0 23.5 5715 13000
[2, 1, 7, 0, 6, 5, 4, 3, 8] 27.0 17.0 6046 14000
[6, 0, 5, 4, 8, 3, 2, 1, 7] 17.0 27.0 6492 15000
[5, 6, 2, 4, 3, 0, 7, 1, 8] 25.0 19.5 6897 16000
[6, 1, 8, 5, 3, 0, 7, 4, 2] 17.0 26.5 7323 17000
[4, 0, 7, 2, 1, 6, 3, 5, 8] 21.0 23.75 7697 18000
[0, 1, 8, 3, 4, 5, 7, 2, 6] 22.0 20.75 8056 19000
[7, 2, 8, 0, 4, 3, 6, 1, 5] 25.0

"There exists no path."

## 3. Conclusion

### 3.1. Comparison of the H-functions

In this section, we are going to use benchmark tests to compare the performace of the different H-functions defined. We will use the examples provided in the previous section to assess performance and compare results. The table below shows the different results found.

The first column shows which H-function was used, followed by a column indicating the example used. The table also shows if the shortest path was found and the memory allocated. The completely investigated points are equal to the number of positions that are on the closed list once the algorithm reaches the goal position. By looking at the number of completely investigated points, we see how goal-oriented the algorithm advanced or how much it spread when searching for the goal. Lastly, we also state how many steps were printed: our algorithm includes an option that allows to choose if every investigated position, or every 100th or 1000th position shall be printed. We sometimes chose a higher value for this option, as otherwise Jupyter reaches its memory capacity for printing output. However, this turns of course a comparison of time and memory across the different examples invalid!

| H-function | Example | Shortest Path Found | Time needed| Completely investigated points|Memory allocated|Step result printing|
| --- | --- | --- |---|---|---|---|
| Hamming Admissible| easy| yes | 3.98 sec|2|157.506 MiB|1|
| Manhattan Admissible| easy | yes |0.39 sec|2|237.475 KiB|1|
| Hamming Weighted Non-Admissible| easy | yes |0.15 sec|2|56.328 KiB|1|
| Manhattan Alternative Non-Admissible| easy | yes | 0.07 sec|2|1.163 MiB|1|
| Manhattan Weighted Non-Admissible| easy | yes | 0.59 sec|2|14.023 MiB|1|
|  | |  |||
| Hamming Admissible| medium| yes | 41.84 sec|1358|67.502 MiB|1|
| Manhattan Admissible| medium | yes |35.15 sec|1330|61.520 MiB|1|
| Hamming Weighted Non-Admissible| medium| yes |3.06 sec|164|2.953 MiB|1|
| Manhattan Alternative Non-Admissible| medium| yes |8.84 sec|387|9.299 MiB|1|
| Manhattan Weighted Non-Admissible| medium | yes| 7.89 sec|249|5.775 MiB|1|
|  | |  |||
| Hamming Admissible| difficult | yes |320.76 sec|~28200|11.692 GiB|100|
| Manhattan Admissible| difficult  | yes |125.64 sec|~17800|4.905 GiB|100|
| Hamming Weighted Non-Admissible| difficult | No (+8 steps longer) |40.46 sec|~9600|1.533 GiB|100|
| Manhattan Alternative Non-Admissible| difficult | No (+2 steps longer) | 3.04 sec|~2200|98.394 MiB|100|
| Manhattan Weighted Non-Admissible| difficult  | No (+4 steps longer)| 42.99 sec|~7600|1007.035 MiB |100|
|  | | |||
| Hamming Admissible| unsolvable| unsolvable |5413.76 sec |~181000|256.535 GiB|1000|
| Manhattan Admissible| unsolvable | unsolvable |5315.34 sec|~181000|255.622 GiB|1000|
| Hamming Weighted Non-Admissible| unsolvable| unsolvable |5230.45 sec|~181000|255.279 GiB|1000|
| Manhattan Alternative Non-Admissible| unsolvable| unsolvable |4601.00 sec|~181000|235.379 GiB|1000|
| Manhattan Weighted Non-Admissible| unsolvable | unsolvable | 6777.63 sec|~181000|263.329 GiB |1000|



#### Benchmark test no. 1: easy example

start1=[1,2,0,3,4,5,6,7,8]

We can see that the algorithm finds the shortest path with all H-functions. However, they do so in a different time manner with the Hamming Admissible function being the slowest and the Manhattan Alternative Non-Admissible being the quickest.

#### Benchmark test no. 2: medium example

start2=[1,3,4,2,7,5,6,8,0]

Here, we can see that A* finds the shorthest path with all H-functions, but there are great differences  with respect to time. The two admissible functions are much slower than the non-admissible versions; most probably because they investigate much more positions. This is due to the fact, that the non-admissible H-functions provide a heavier weight in comparison the the G-function. This leads the algorithm to search more goal-oriented. However, this also opens the door to the algorithm possibly missing the shortest path; a problem conveyed in the following example.

We include a ~ sign to point out that here and in the following example the number of completly investigated points is not exact. This is due to the fact that only every full 100th (1000th below) position is printed.

#### Benchmark test no. 3:  difficult example

start3 = [8,1,7,4,5,6,2,0,3] <br>


We can see that the admissible H-functions are much slower than the non-admissible ones and they find the optimal shorthest path as expected. Even though the non-admissible versions are significantly quicker, they fail to find the shorthest path, which is a severe drawback. It is the Manhattan Alternative Non-Admissible function that finds the path which is the closest to the optimal (it is only 2 steps longer) and in return, it is also the quickest one out of all versions. 

Comparing our two admissible functions, we can see that the Manhattan admissible is much quicker and investigates much fewer points than the Hamming admissible function. This is in line with the results from the other examples. However, here this difference is much more striking with the former taking only 39% of the time of the later. Based on this, we can conclude that the Manhattan distance is the best admissible function we have. 

In general, this example points out clearly the trade-off between speed and optimality that we face when choosing a well-performing H-function. Also the results are in line with everything what we expected from our admissible and non-admissible H-functions.

#### Benchmark test no. 4: unsolvable example

start4 = [4,6,3,5,7,1,2,8,0] <br>

This example is impossible to be solved by definition. Our A* algorithm succefully concludes on the non-existence of a path to the goal regardless of the H-function used. The different H-functions are investigating the same amount of positions (the total possible positions are 9!, which half of them being unsolvale with the given goal). They also use a similar memory allocated. However, they differ by as much as 36 minutest to come to this conclusion. This is due to the complexity of the computation method and the respective time it needs to control each position.

#### Final conclusion

We can see that the H-function controls the behaviour of A*:

 - In an extreme case, where $ h(n) =0 \;\; \forall n$, only the G-value plays a role and A* turns into the Dijkstra's algorithm.

 - An H-function where the cost of moving is always lower or equal to the actual cost guarantees that the shorthest path is found, but it may be very slow in some cases (this is the case with our Hamming Admissible function). The lower the estimated cost to reach the goal is in comparison the the cost of reaching this node, the more nodes A* will expand making the algorithm slower.

 - An H-function that can estimate the cost with 100% accuracy assures that A* follows the best path and never expand anything else, which makes it very fast. As such a function is also admissible, the shorthest path is always found. Unfortunately, it is not always possible to produce a perfectly accurate H-function.

 - An H-function that overestimates the cost is not guaranteed to find the shortest path, but it finds a path much more quickly (Hamming Weighted Non-Admissible, Manhattan Alternative Non-Admissible, Manhattan Weighted Non-Admissible in our case). 

 - An interesting point is the extreme case where the estimated cost is very low relative to the G-value, A* turns into Greedy Best-First-Search algorithm.

Every A* algorithm is as good as its H-function is: this is the core element of A*. To choose which H-function is optimal to use depends on the purpose. In some cases, it is enough to find a "good" path instead of a "perfect" path which means we can speed up the search and relax the admissibility criterion. Depending on what we want, we can always adjust the H-function accordingly which is a very useful property of the algorithm.

### 3.2. Further extension possibilities

The A* algorithm has plenty of variants which can be also interesting to investigate and which can be an interesting extension to this present notebook. For instance, the anytime repairing A* (ARA* ) which generates a fast, non-optimal solution. The difference between A* and ARA* is that the latter adds a time limit to the algorithm. It was developed because the optimal A* is too slow for many purposes. ARA* is very quick in finding a first, most probably highly sub-optimal solution and then it works on improving the solution until it reaches its time limit.

The A* algorithm inspired different classes of incremental heuristic search algorithms which are used to solve sequences of similar search problems. These algorithms are mainly used on domains that dynamically change or are not completely known. In general, incremental search algorithms reuse information from previous searches to speed up the current search. We can distinguish three main classes of incremental heuristic algorithms:

 - First class: restarts A* at the point where its current search deviates from the previous one (i.e. Fringe Saving A* (FSA*)).
 - Second class: updates the H-values from the previous search during the current search to make them more informed (i.e. Generalized Adaptive A* (GAA*)).
 - Third class: updates the G-values from the previous search during the current search to correct them when necessary (i.e. Lifelong Planning A* (LPA*)).
 






##  Sources

Chandrashekar T.S., Narahari Y. (2005) Economic Mechanisms for Shortest Path Cooperative Games with Incomplete Information. In: Deng X., Ye Y. (eds) Internet and Network Economics. WINE 2005. Lecture Notes in Computer Science, vol 3828. Springer, Berlin, Heidelberg

Chen, B. Y., Lam, W. H., Sumalee, A., Li, Q., Shao, H., & Fang, Z. (2013). Finding reliable shortest paths in road networks under uncertainty. Networks and spatial economics, 13(2), 123-148.

Cormen, T. H., Leiserson, C. E., Rivest, R. L., & Stein, C. (2009). Introduction to algorithms. MIT press.

Matthew O. Jackson, Brian W. Rogers (2005), The Economics of Small Worlds, Journal of the European Economic Association, Volume 3, Issue 2-3, Pages 617–627

Shortest path problem from QuantEcon: https://julia.quantecon.org/dynamic_programming/short_path.html

A useful source for the heuristics: http://theory.stanford.edu/~amitp/GameProgramming/Heuristics.html